In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src import matplotlib_style  # noqa
import matplotlib.pyplot as plt

import src.assumptions as A
from src import supply_model

from src.data import demand
from src.units import Units as U
import pandas as pd
from src import storage_model

from pathlib import Path

# days without excess supply plots

In [ ]:
# reproduce Rei's plot
A.Nuclear.Capacity = 12 * U.GW
days_without_excess_nuclear = supply_model.fraction_days_without_excess("era5", return_mean=False)
A.Nuclear.Capacity = 0 * U.GW
days_without_excess_none = supply_model.fraction_days_without_excess("era5", return_mean=False)

plt.figure()
plt.plot(days_without_excess_none, days_without_excess_none.index.values, label="0 GW Nuclear")
plt.plot(days_without_excess_nuclear, days_without_excess_nuclear.index.values, label="12 GW Nuclear")
plt.xlabel("Days without Excess Generation")
plt.ylabel("Renewable Capacity (GW)")
plt.ylim(0, 500)
plt.legend()

In [ ]:
# my version
A.Nuclear.Capacity = 12 * U.GW
era5_nuclear = supply_model.fraction_days_without_excess("era5", return_mean=True)
espeni_nuclear = supply_model.fraction_days_without_excess("espeni", return_mean=True)
A.Nuclear.Capacity = 0 * U.GW
era5_no_nuclear = supply_model.fraction_days_without_excess("era5", return_mean=True)
espeni_no_nuclear = supply_model.fraction_days_without_excess("espeni", return_mean=True)

plt.figure()
plt.plot(era5_nuclear.index.values, era5_nuclear, label="ERA5 12 GW Nuclear")
plt.plot(era5_no_nuclear.index.values, era5_no_nuclear, label="ERA5 0 GW Nuclear")
plt.plot(espeni_nuclear.index.values, espeni_nuclear, ls="--", label="ESPENI 12 GW Nuclear", color="blue")
plt.plot(espeni_no_nuclear.index.values, espeni_no_nuclear, ls="--", label="ESPENI 0 GW Nuclear", color="orange")
plt.xlabel("Renewable Capacity (GW)")
plt.ylabel("Days without Excess Generation")
plt.legend()

In [ ]:
# now a version comparing naive and new demand scaling
A.Nuclear.Capacity = 12 * U.GW
naive_nuclear = supply_model.fraction_days_without_excess("era5", return_mean=True, naive_demand_scaling=True)
new_nuclear = supply_model.fraction_days_without_excess("era5", return_mean=True, naive_demand_scaling=False)
A.Nuclear.Capacity = 0 * U.GW
naive_no_nuclear = supply_model.fraction_days_without_excess("era5", return_mean=True, naive_demand_scaling=True)
new_no_nuclear = supply_model.fraction_days_without_excess("era5", return_mean=True, naive_demand_scaling=False)

plt.figure()
plt.plot(naive_nuclear.index.values, naive_nuclear, label="Naive 12 GW Nuclear")
plt.plot(naive_no_nuclear.index.values, naive_no_nuclear, label="Naive 0 GW Nuclear")
plt.plot(new_nuclear.index.values, new_nuclear, ls="--", label="Seasonal 12 GW Nuclear", color="blue")
plt.plot(new_no_nuclear.index.values, new_no_nuclear, ls="--", label="Seasonal 0 GW Nuclear", color="orange")
plt.xlabel("Renewable Capacity (GW)")
plt.ylabel("Days without Excess Generation")
plt.legend()

# storage model plots

In [ ]:
renewable_capacity = 350
df = supply_model.get_net_supply().reset_index()

storage = storage_model.StorageModel(
    renewable_capacity=renewable_capacity * U.GW,
    max_storage_capacity=A.HydrogenStorage.CavernStorage.MaxCapacity,
    electrolyser_power=A.HydrogenStorage.Electrolysis.Power,
    dac_capacity=A.DAC.Capacity,
)
net_supply_df = storage.run_simulation(df)
results = storage.analyze_simulation_results(net_supply_df)
storage.print_simulation_results(results)

# meet L>20
plt.figure(figsize=(7, 4), dpi=200)
plt.plot(net_supply_df[f"L (TWh),RC={renewable_capacity}GW"], color="g", linewidth=0.3, label="Energy in Storage")
plt.ylim(0, storage.max_storage_capacity * 1.1)
plt.xlabel("Day in 40 years")
plt.ylabel("Stored Energy (TWh)")
plt.axhline(storage.max_storage_capacity, linestyle="dashed", color="r", label="Minimum Storage Capacity")
plt.axhline(20, linestyle="dashed", color="b", label="Threshold Energy in Storage")
plt.legend()


In [ ]:
df = supply_model.get_net_supply().reset_index()
# plot a 3D surface of x (renewable capacity), y (electrolyser power) and z (minimum storage)
renewable_capacities = range(350, 500, 10)
electrolyser_powers = range(20, 110, 10)
xyzs = []
for renewable_capacity in renewable_capacities:
    for electrolyser_power in electrolyser_powers:
        model = storage_model.StorageModel(
            renewable_capacity=renewable_capacity * U.GW,
            max_storage_capacity=A.HydrogenStorage.CavernStorage.MaxCapacity,
            electrolyser_power=electrolyser_power * U.GW,
            dac_capacity=A.DAC.Capacity,
        )

        net_supply_df = model.run_simulation(df.copy())
        results = model.analyze_simulation_results(net_supply_df)
        xyz = (renewable_capacity, electrolyser_power, results["minimum_storage"].magnitude)
        xyzs.append(xyz)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8, 6), dpi=200)
ax = fig.add_subplot(111, projection="3d")

# Prepare data for 3D surface plot
xyz = np.array(xyzs)
X = xyz[:, 0].reshape(len(renewable_capacities), len(electrolyser_powers))
Y = xyz[:, 1].reshape(len(renewable_capacities), len(electrolyser_powers))
Z = xyz[:, 2].reshape(len(renewable_capacities), len(electrolyser_powers))

surf = ax.plot_surface(X, Y, Z, cmap="viridis", alpha=0.9)
ax.set_xlabel("Renewable Capacity (GW)")
ax.set_ylabel("Electrolyser Power (GW)")
ax.set_zlabel("Minimum Storage (TWh)")
ax.set_title("3D Surface: Minimum Storage vs Renewable Capacity & Electrolyser Power")

# Set the viewing angle
ax.view_init(elev=30, azim=210)

# Add a color bar
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
# make a similar plot using plotly for interactivity
import plotly.graph_objects as go

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale="Viridis")])
fig.update_layout(
    title="3D Surface: Minimum Storage vs Renewable Capacity & Electrolyser Power",
    scene=dict(
        xaxis_title="Renewable Capacity (GW)",
        yaxis_title="Electrolyser Power (GW)",
        zaxis_title="Minimum Storage (TWh)",
    ),
)
fig.show()